# Get tweets with given hashtag from Twitter

For a user-specified hashtag, grabs tweets from the Twitter API and dumps them once an hour into a data folder. Also unfolds the user and entities part of the returned data, to give you nice clean data columns. 

This code will run over an hour at a time, then you'll need to rerun the last cell to get more data. A better way to do this is to run the code from a cron job, as a .py file

In [10]:
# Set up the link to Twitter
import twitter
import pandas as pd
import time
import datetime
import re


fsecret = open('../../../../../../twittersecrets.txt', 'r')
secrets = fsecret.readline()
ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET = secrets.strip().split(',')
ACCESS_TOKEN = ACCESS_TOKEN.strip()
ACCESS_TOKEN_SECRET = ACCESS_TOKEN_SECRET.strip()
CONSUMER_KEY = CONSUMER_KEY.strip()
CONSUMER_SECRET = CONSUMER_SECRET.strip()

auth = twitter.oauth.OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
ta = twitter.Twitter(auth=auth)

In [11]:
# Get your first page of data
hashtag = '#qanon'
tweetjson = ta.search.tweets(q=hashtag)
dftweet = pd.DataFrame(tweetjson['statuses'])
dftweet = pd.concat([dftweet, dftweet['user'].apply(pd.Series).add_prefix('user_'),
                     dftweet['entities'].apply(pd.Series).add_prefix('entity_')], axis=1)
maxid = dftweet['id'].iloc[-1]-1 #ungodly hack because I haven't had enough coffee - google!
dftweet

TwitterHTTPError: Twitter sent status 429 for URL: 1.1/search/tweets.json using parameters: (oauth_consumer_key=ZaEQOpmkTALz9qg0WrAzCQ&oauth_nonce=6286389233958448532&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1522653784&oauth_token=42722682-3I4FBxLgICuYL93mQpttUThQSsAJfW23Di2wMgDvU&oauth_version=1.0&q=%23qanon&oauth_signature=cWWEiIFqdesU6%2FZYor%2BpAEAgVf4%3D)
details: {'errors': [{'message': 'Rate limit exceeded', 'code': 88}]}

In [ ]:
# Now work backwards in time, grabbing more data each time
# This bit of code should take an hour to run... and then dump out a timestamped file.
#FIXIT - escape out carriage returns in the free-text fields
for j in range(300):
    for i in range(180):
        try:
            print('{}: grabbing tweets backwards from {}'.format(i, maxid))
            tweetjson = ta.search.tweets(q=hashtag, max_id=maxid)
            dftweet = dftweet.append(pd.DataFrame(tweetjson['statuses']))
            minid = dftweet['id'].iloc[0]
            maxid = dftweet['id'].iloc[-1]-1
            time.sleep(1)
        except:
            print('Dammit, twitter timed us out')
            break
            #time.sleep(10)

    #Send this batch of tweets to a CSV file
    if len(dftweet) > 0:
        timestamp = re.sub('[^\d]+', '_', str(datetime.datetime.now()))
        outfile = '../data/{}_tweets_{}_{}_{}.csv'.format(hashtag, timestamp, 
                                                          minid, maxid)
        dftweet = pd.concat([dftweet, dftweet['user'].apply(pd.Series).add_prefix('user_'),
                             dftweet['entities'].apply(pd.Series).add_prefix('entity_')], axis=1)
        dftweet.to_csv(outfile, index=False)
        print('{} rows sent to file {}'.format(len(dftweet), outfile))
        print(dftweet[['id', 'created_at']].head())
        dftweet = pd.DataFrame([])
    else:
        print("No tweets found")

    print('Run {} complete. Napping for 15 mins'.format(j))
    time.sleep(15*60)

0: grabbing tweets backwards from 980228029749309439
1: grabbing tweets backwards from 980227969544224768
2: grabbing tweets backwards from 980227906856202239
3: grabbing tweets backwards from 980227852720361477
4: grabbing tweets backwards from 980227799658258431
5: grabbing tweets backwards from 980227710550192128
6: grabbing tweets backwards from 980227646058651647
7: grabbing tweets backwards from 980227585060818943
8: grabbing tweets backwards from 980227514206474239
9: grabbing tweets backwards from 980227449765138432
10: grabbing tweets backwards from 980227398234001407
11: grabbing tweets backwards from 980227321578774527
12: grabbing tweets backwards from 980227258769027071
13: grabbing tweets backwards from 980227186937352191
14: grabbing tweets backwards from 980227129114800128
15: grabbing tweets backwards from 980227041256792063
16: grabbing tweets backwards from 980226988383330303
17: grabbing tweets backwards from 980226930078253056
18: grabbing tweets backwards from 980

151: grabbing tweets backwards from 980217889771831296
152: grabbing tweets backwards from 980217824730800128
153: grabbing tweets backwards from 980217754631507968
154: grabbing tweets backwards from 980217668279169024
155: grabbing tweets backwards from 980217610724921344
156: grabbing tweets backwards from 980217569951989759
157: grabbing tweets backwards from 980217501035454473
158: grabbing tweets backwards from 980217412325969919
159: grabbing tweets backwards from 980217356671692799
160: grabbing tweets backwards from 980217312736243712
161: grabbing tweets backwards from 980217245119823872
162: grabbing tweets backwards from 980217204103933957
163: grabbing tweets backwards from 980217143848562687
164: grabbing tweets backwards from 980217088781508609
165: grabbing tweets backwards from 980217030958821375
166: grabbing tweets backwards from 980216980316803071
167: grabbing tweets backwards from 980216941884428287
168: grabbing tweets backwards from 980216877808078848
169: grabb

114: grabbing tweets backwards from 980208476931620863
115: grabbing tweets backwards from 980208425421459455
116: grabbing tweets backwards from 980208382119432193
117: grabbing tweets backwards from 980208332018409471
118: grabbing tweets backwards from 980208239873855487
119: grabbing tweets backwards from 980208148861550593
120: grabbing tweets backwards from 980208080666456064
121: grabbing tweets backwards from 980208008218148864
122: grabbing tweets backwards from 980207947912499200
123: grabbing tweets backwards from 980207868275171328
124: grabbing tweets backwards from 980207778282295297
125: grabbing tweets backwards from 980207690378039295
126: grabbing tweets backwards from 980207585667223553
127: grabbing tweets backwards from 980207519237775359
128: grabbing tweets backwards from 980207448635011071
129: grabbing tweets backwards from 980207383610671103
130: grabbing tweets backwards from 980207313394065407
131: grabbing tweets backwards from 980207244737499136
132: grabb

76: grabbing tweets backwards from 980198345573507071
77: grabbing tweets backwards from 980198258587914244
78: grabbing tweets backwards from 980198170750849024
79: grabbing tweets backwards from 980198074818682880
80: grabbing tweets backwards from 980198008246718464
81: grabbing tweets backwards from 980197935911649279
82: grabbing tweets backwards from 980197852881215487
83: grabbing tweets backwards from 980197812569767935
84: grabbing tweets backwards from 980197696102420479
85: grabbing tweets backwards from 980197607082409983
86: grabbing tweets backwards from 980197540632186880
87: grabbing tweets backwards from 980197427708866563
88: grabbing tweets backwards from 980197354715271167
89: grabbing tweets backwards from 980197289452101631
90: grabbing tweets backwards from 980197209269350399
91: grabbing tweets backwards from 980197156379340799
92: grabbing tweets backwards from 980197073311207423
93: grabbing tweets backwards from 980196978058395648
94: grabbing tweets backward

38: grabbing tweets backwards from 980187019468066817
39: grabbing tweets backwards from 980186945165836288
40: grabbing tweets backwards from 980186870385725440
41: grabbing tweets backwards from 980186803763367935
42: grabbing tweets backwards from 980186704362557439
43: grabbing tweets backwards from 980186614302412799
44: grabbing tweets backwards from 980186540444995589
45: grabbing tweets backwards from 980186489375154175
46: grabbing tweets backwards from 980186430663286783
47: grabbing tweets backwards from 980186371699585023
48: grabbing tweets backwards from 980186303454117887
49: grabbing tweets backwards from 980186258315137023
50: grabbing tweets backwards from 980186190409355263
51: grabbing tweets backwards from 980186111593955327
52: grabbing tweets backwards from 980186041444372481
53: grabbing tweets backwards from 980185978437603329
54: grabbing tweets backwards from 980185902038339584
55: grabbing tweets backwards from 980185802008334336
56: grabbing tweets backward

...

175: grabbing tweets backwards from 979204559875829759
176: grabbing tweets backwards from 979204508524793855
177: grabbing tweets backwards from 979204415725830145
178: grabbing tweets backwards from 979204362135195647
179: grabbing tweets backwards from 979204324957065216
2658 rows sent to file ../data/#qanon_tweets_2018_04_04_06_35_37_726943_979215191668920321_979204292631392255.csv
                   id                      created_at
0  979215191668920321  Thu Mar 29 04:34:29 +0000 2018
1  979215187524902913  Thu Mar 29 04:34:28 +0000 2018
2  979215182332420096  Thu Mar 29 04:34:27 +0000 2018
3  979215180683870208  Thu Mar 29 04:34:26 +0000 2018
4  979215178448457729  Thu Mar 29 04:34:26 +0000 2018
Run 84 complete. Napping for 15 mins
0: grabbing tweets backwards from 979204292631392255
1: grabbing tweets backwards from 979204209525444607
2: grabbing tweets backwards from 979204139493203967
3: grabbing tweets backwards from 979204044844650497
4: grabbing tweets backwards from 9792

138: grabbing tweets backwards from 979197290056376319
139: grabbing tweets backwards from 979197257672282113
140: grabbing tweets backwards from 979197217448779776
141: grabbing tweets backwards from 979197188470378495
142: grabbing tweets backwards from 979197140911185919
143: grabbing tweets backwards from 979197088746663935
144: grabbing tweets backwards from 979197049102065663
145: grabbing tweets backwards from 979197001207345151
146: grabbing tweets backwards from 979196951575973887
147: grabbing tweets backwards from 979196905757528063
148: grabbing tweets backwards from 979196855694196736
149: grabbing tweets backwards from 979196791110406143
150: grabbing tweets backwards from 979196747124654080
151: grabbing tweets backwards from 979196677457399807
152: grabbing tweets backwards from 979196620301656063
153: grabbing tweets backwards from 979196574113914879
154: grabbing tweets backwards from 979196514919731199
155: grabbing tweets backwards from 979196475237453823
156: grabb

101: grabbing tweets backwards from 979191488415502335
102: grabbing tweets backwards from 979191444845027327
103: grabbing tweets backwards from 979191410674012159
104: grabbing tweets backwards from 979191350775214080
105: grabbing tweets backwards from 979191288309387263
106: grabbing tweets backwards from 979191245674344447
107: grabbing tweets backwards from 979191212434427903
108: grabbing tweets backwards from 979191180901732352
109: grabbing tweets backwards from 979191145506004992
110: grabbing tweets backwards from 979191106436034564
111: grabbing tweets backwards from 979191061171064831
112: grabbing tweets backwards from 979191042628116479
113: grabbing tweets backwards from 979190996880650242
114: grabbing tweets backwards from 979190974520868864
115: grabbing tweets backwards from 979190933639057412
116: grabbing tweets backwards from 979190894543949823
117: grabbing tweets backwards from 979190850466000896
118: grabbing tweets backwards from 979190821495848959
119: grabb

63: grabbing tweets backwards from 979184902259093503
64: grabbing tweets backwards from 979184854297337855
65: grabbing tweets backwards from 979184797657518079
66: grabbing tweets backwards from 979184756775702529
67: grabbing tweets backwards from 979184707349958655
68: grabbing tweets backwards from 979184660092805119
69: grabbing tweets backwards from 979184612072189954
70: grabbing tweets backwards from 979184551716118528
71: grabbing tweets backwards from 979184511144558591
72: grabbing tweets backwards from 979184450226540544
73: grabbing tweets backwards from 979184396719685631
74: grabbing tweets backwards from 979184350817341439
75: grabbing tweets backwards from 979184326167449600
76: grabbing tweets backwards from 979184293170827264
77: grabbing tweets backwards from 979184246974795775
78: grabbing tweets backwards from 979184204649934848
79: grabbing tweets backwards from 979184169879302143
80: grabbing tweets backwards from 979184126782857218
81: grabbing tweets backward

25: grabbing tweets backwards from 979178388278591487
26: grabbing tweets backwards from 979178343219286015
27: grabbing tweets backwards from 979178283999866879
28: grabbing tweets backwards from 979178239162748927
29: grabbing tweets backwards from 979178194128527366
30: grabbing tweets backwards from 979178146997194752
31: grabbing tweets backwards from 979178111681081344
32: grabbing tweets backwards from 979178073429078015
33: grabbing tweets backwards from 979178015853867007
34: grabbing tweets backwards from 979177973910827007
